In [35]:
from tqdm import tqdm 
import pandas as pd
import os

from elasticsearch import Elasticsearch, helpers

## Evaluate Retrieval

### Text Evalition

#### Create index on documents

In [2]:
# Read data into dataframe 
data_file_path = os.path.join('..','app','data', 'chunked_data.csv')
df = pd.read_csv(data_file_path).dropna()

# Convert dataframe to list of dictionaries
data_dict = df.to_dict(orient="records")

In [3]:
embedding_size = 384

In [5]:
es = Elasticsearch('http://localhost:9200') 

index_settings={
    "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
        "title": {"type": "keyword"},
        "transcript": {"type": "text"},
        "topics": {"type": "text"},
        "id": {"type": "keyword"},}
    }
}
index_name = 'ted-rag'

In [14]:
# es.indices.delete(index=index_name,ignore_unavailable=True)
# es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-qa-index'})

In [15]:
# Add Data to Index using index()
for i in tqdm(range(len(data_dict))):
    row = data_dict[i]
    es.index(index=index_name, id=i, document=row)

  0%|          | 0/639 [00:00<?, ?it/s]

100%|██████████| 639/639 [00:39<00:00, 16.17it/s]


#### Define elastic search function

In [14]:
def elastic_search(query, title = None):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["transcript^3", "title", "topics"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "title": title
                #     }
                # }
            }
        }
    }
    response = es.search(index=index_name, body=search_query)
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

#### Get ground truth data

In [10]:
ground_truth_file_path = os.path.join('ground_truth.csv')
df_ground_truth = pd.read_csv(ground_truth_file_path, header = 0, usecols=[0, 1])

In [11]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [12]:
print(len(ground_truth), ground_truth[0])

258 {'question': 'What topic does Hans Rosling primarily address in his talk?', 'id': 0}


In [15]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['id']
    results = elastic_search(query=q['question'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████| 258/258 [00:16<00:00, 15.25it/s]


#### Calucalte hit rate and mrr

In [16]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [17]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [18]:
(hit_rate(relevance_total), mrr(relevance_total)) # question3, answer3

(0.6434108527131783, 0.5498062015503875)

Elastic Search 
```
(0.64 , 0.54)
```

#### Using minisearch

In [21]:
import sys
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
# Add the app directory to the Python path
app_dir = os.path.join(notebook_dir, '../app')
sys.path.append(app_dir)

In [25]:
import src.minisearch as minisearch
ms = minisearch.Index(
    text_fields=['transcript', 'topics'],
    keyword_fields=['title','id'],
)

ms.fit(data_dict)

In [26]:
def mini_search(query):
    results = ms.search( query = query,
                    num_results = 5)
    # response = [result['answer'] for result in results]
    # return response
    return results

In [27]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['id']
    results = mini_search(query=q['question'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████| 258/258 [00:00<00:00, 359.72it/s]


In [28]:
(hit_rate(relevance_total), mrr(relevance_total))

(0.5658914728682171, 0.4399870801033592)

minisearch 
```
(0.56, 0.43)
```

### Vector Evalution

In [81]:
from src.constants import embedding_size, HF_API_TOKEN
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

In [82]:
emb_model = HuggingFaceInferenceAPIEmbeddings(
        api_key=HF_API_TOKEN, 
        model_name="sentence-transformers/all-MiniLM-l6-v2"
    )
index_name = 'ted-eval'

In [32]:
index_settings={
    "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
        "transcript": {"type": "text"},
        "topics": {"type": "text"},
        "title": {"type": "text"},
        "id": {"type": "text"},
        "transcript_vector": {"type": "dense_vector", "dims": embedding_size, "index": True, "similarity": "cosine"},
        "title_transcript_vector": {"type": "dense_vector", "dims": embedding_size, "index": True, "similarity": "cosine"},        }
    }
}

# Create Index and delete if it already exists
es.indices.delete(index=index_name, ignore_unavailable=True)
es.indices.create(index=index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ted-eval'})

Embedding the answer and query vector

In [33]:
batch_size = 100
def get_embeddings_batch( texts):
    """
    Generates embedding vectors for a batch of texts using the Hugging Face endpoint.
    """
    data = emb_model.embed_documents(texts)[:embedding_size]
    return data 

In [34]:
vector_data_dict = []
for i in tqdm(range(0, len(data_dict), batch_size)):
    batch = data_dict[i:i + batch_size]
    title_transcripts = [f"{data['title']} {data['transcript']}" for data in batch]
    transcripts = [data['transcript'] for data in batch]
    transcript_embeddings = get_embeddings_batch(transcripts)
    title_transcript_embeddings = get_embeddings_batch(title_transcripts)
    
    for j, data in enumerate(batch):
        data['transcript_vector'] = transcript_embeddings[j][:embedding_size]
        data['title_transcript_vector'] = title_transcript_embeddings[j][:embedding_size]
        vector_data_dict.append(data)

100%|██████████| 23/23 [00:31<00:00,  1.36s/it]


Adding data to index

In [36]:

# Add Data to Index using index() 
actions = [
    {
        "_index": index_name,
        "_id": i,
        "_source": row
    }
    for i, row in enumerate(vector_data_dict)
]
helpers.bulk(es, actions)

(2290, [])

In [37]:
print(es.info())

{'name': '58ce424d0117', 'cluster_name': 'docker-cluster', 'cluster_uuid': '0F660e46RlOZvAeyTjObFw', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [38]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 3,
        "num_candidates": 100,
    }
    search_query = {
        "knn": knn,
        "_source": ["transcript", "title", "topic", "id"]
    }
    es_results = es.search(
        index=index_name, 
        body=search_query
    )
    result_docs = []
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

#### Transcript vector

In [50]:
def transcript_vector_knn(q):
    question = q['question'] 

    v_q = emb_model.embed_query(question)[:embedding_size]

    return elastic_search_knn('transcript_vector', v_q)

In [51]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance) 
    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [52]:
evaluate(ground_truth, transcript_vector_knn)

100%|██████████| 258/258 [01:05<00:00,  3.94it/s]


{'hit_rate': 0.5542635658914729, 'mrr': 0.4651162790697673}

Transcript Vector
```
(0.55, 0.46)
```

#### Transcript title vector

In [55]:
def title_transcript_vector_knn(q):
    question = q['question'] 

    v_q = emb_model.embed_query(question)[:embedding_size]

    return elastic_search_knn('title_transcript_vector', v_q)

In [56]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [57]:
evaluate(ground_truth, title_transcript_vector_knn)

100%|██████████| 258/258 [00:57<00:00,  4.47it/s]


{'hit_rate': 0.562015503875969, 'mrr': 0.46576227390180863}

Title Transcript Vector
```
(0.56, 0.46)
```

### Hybrid Search Evaluation

#### Create search function

In [84]:
def hybrid_elastic_search(field, query, query_vector):
    keyword_query = {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["transcript^3", "title", "topics"],
                        "type": "best_fields",
                        "boost": 0.6
                    },
                },
            }
        }
    knn_query = {
        "field": field,
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100,
        "boost": 0.4
    }
    search_query = {
        "knn": knn_query,
        "query": keyword_query,
        "_source": ["transcript", "title", "topics", "id"],
        "size":5
    }
    es_results = es.search(
        index=index_name, 
        body=search_query
    )
    result_docs = []
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

#### Title-transcript vector

In [85]:
def transcript_hybrid(q):
    query = q['question'] 
    v_q = emb_model.embed_query(query)[:embedding_size]

    return hybrid_elastic_search('title_transcript_vector', query,v_q)

In [86]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance) 
    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [87]:
evaluate(ground_truth, transcript_hybrid)

  0%|          | 0/258 [00:00<?, ?it/s]

What topic does Hans Rosling primarily address in his talk?


  0%|          | 1/258 [00:00<02:45,  1.56it/s]

How did Hans Rosling assess his students' knowledge of child mortality?


  1%|          | 3/258 [00:01<01:23,  3.06it/s]

...

What are the key recommendations for improving sleep quality?


100%|█████████▉| 257/258 [01:48<00:00,  3.72it/s]

What is the main message about the necessity of sleep?


100%|██████████| 258/258 [01:49<00:00,  2.36it/s]


{'hit_rate': 0.6434108527131783, 'mrr': 0.5166666666666665}

Best Performancing Retrieval:

Elastic Search with Question-Answer vector 
```
hit rate: 0.87, mrr: 0.85
```